<a href="https://colab.research.google.com/github/jaybhuvaa/LLM/blob/main/Bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
with open('woo.txt','r',encoding='utf-8') as f:
    text=f.read()
print(len(text))
print(text[:200])


232131
﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


**Tokenizer**

Tokenization in Python is essentially splitting a phrase, sentence, paragraph, or an entire text document into smaller units, such as individual words or terms. Each of these smaller units are called tokens. The tokens could be words, numbers or punctuation marks.

In [14]:
chars=sorted(set(text))
print(chars)
print(len(chars))


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


**Encoding-Deconding**

string_to_int = {ch:i for i , ch in enumerate (chars)}:

This line creates a dictionary string_to_int where each character in the chars string is mapped to its corresponding index in the chars string. The enumerate() function provides both the index and the character from the chars string.

encode=lambda s:[string_to_int[c] for c in s]:

This line defines an anonymous function encode which takes a string s as input. It uses a list comprehension to iterate over each character c in the input string s, and for each character, it retrieves its corresponding integer value from the string_to_int dictionary and forms a list of these integer values.

In [15]:
string_to_int = {ch:i for i , ch in enumerate (chars)}
int_to_string = {i:ch for i , ch in enumerate (chars)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:''.join([int_to_string[i] for i in l])

print(encode('Hello'))
print(encode('Jay'))
print(decode([32, 58, 65]))

[32, 58, 65, 65, 68]
[34, 54, 78]
Hel


In [16]:
message1 = "The quick brown fox jumps over the lazy dog."
message2 = "A fast brown fox leaps over a sleepy dog."

emsg1 = encode(message1)
emsg2 = encode(message2)

print(type(emsg1))


<class 'list'>


**PyTorch**

PyTorch is an open source machine learning (ML) framework based on the Python programming language and the Torch library. Torch is an open source ML library used for creating deep neural networks and is written in the Lua scripting language. It's one of the preferred platforms for deep learning research. The framework is built to speed up the process between research prototyping and deployment.

In [17]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


**Test-Validation Split**

> Why to split data?

  if we give model our full data to train than there is no any
    prediction part. it will be only factual data that will be given by model.
    
 if we split the data than model will be train based on the taining data anf to test the accuracy we can use the test data to check weather the model is doing right or wrong.
  



In [18]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
# print(train_data[:10])
# print(val_data[:10])

def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

tensor([161468,  40843, 134403, 125247])
inputs:
torch.Size([4, 8])
tensor([[73,  1, 61, 58, 71,  1, 62, 67],
        [ 9,  3,  1, 72, 54, 62, 57,  1],
        [68, 76,  1, 76, 54, 72,  1, 65],
        [72, 73, 71, 58, 73, 56, 61, 58]])
targets:
torch.Size([4, 8])
tensor([[ 1, 61, 58, 71,  1, 62, 67,  1],
        [ 3,  1, 72, 54, 62, 57,  1, 73],
        [76,  1, 76, 54, 72,  1, 65, 62],
        [73, 71, 58, 73, 56, 61, 58, 57]])


**Bigram Model**

A bigram language model is a type of statistical language model that predicts the probability of a word in a sequence based on the previous word. It considers pairs of consecutive words (bigrams) and estimates the likelihood of encountering a specific word given the preceding word in a text or sentence.

In [19]:
block_size=8
batch_size=4

x=train_data[:block_size+1]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([80]) the target: 1
when input is tensor([80,  1]) the target: 1
when input is tensor([80,  1,  1]) the target: 28
when input is tensor([80,  1,  1, 28]) the target: 39
when input is tensor([80,  1,  1, 28, 39]) the target: 42
when input is tensor([80,  1,  1, 28, 39, 42]) the target: 39
when input is tensor([80,  1,  1, 28, 39, 42, 39]) the target: 44
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) the target: 32


cuda is used for gpu based computation or training purpose.

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [25]:
import torch.nn as nn
import torch.nn.functional as F
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(len(chars))
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


U﻿Tr.WcnXR:02s863kzC]3'h JnwW"fPgN6e﻿Hg
34Tq-4ZhRN&fu7
aqa﻿(([,p!,S?lO?)6e?*eU(i
kFh-XWF0Uk0H j,o)T3'Mj-4&XHvNv_BBovYPcRi8eEE-z
kl&0,Qd&
r_49)MFexSWx.B:V&N.Ezp;gjtR2:aaQ OCaajRa!F_-sa5hb6Bf﻿(dbZr)6aLKGVLR:0myf!Dq:WoT?tv7Upu u;(icKg?*99FZSDl)"iU!r'﻿s]n:0v
)lfH9*,GMkis(KCyuW4L'&8.rco)Jkz29ynOEDMUJlX9Ks y!﻿G3Pghv!mpuXAfCIz(anXZ(d"FEza5zDVD:MFE3vBMzH1H;x 9v0uX"P)6xKJmOR[,(4SKGS"Q )jD[YOuDl2psLAGMZ,U!q_K6JybYWK7Z!V(vGUA0YKcwd6Jl"7OCS"Qyo,4qZNSmL:3,6e?*?myTVJ:0xp;!-Boo?v'W8y﻿7wiUgBoz2HX"9xVRaFG2R6po4]
